In [23]:
from pyDeepInsight import ImageTransformer, LogScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

from matplotlib import pyplot as plt
import seaborn as sns

import itertools


In [2]:
prosody_data = pd.read_csv('Prosodic_Features_MRH.csv',sep=",")
#expr = pd.read_csv(expr_file, sep="\t")
y = prosody_data['Hired3C'].values
#36
#X =prosody_data[['duration', 'energy', 'power', 'min_pitch', 'max_pitch', 'mean_pitch', 'pitch_sd', 'pitch_abs', 'pitch_quant', 'pitchUvsVRatio', 'Time:8', 'iDifference', 'diffPitchMaxMin', 'diffPitchMaxMean', 'diffPitchMaxMode', 'intensityMin', 'intensityMax', 'intensityMean', 'intensitySD', 'intensityQuant', 'diffIntMaxMin', 'diffIntMaxMean', 'diffIntMaxMode', 'avgVal1', 'avgVal2', 'avgVal3', 'avgBand1', 'avgBand2', 'avgBand3', 'fmean1', 'fmean2', 'fmean3', 'f2meanf1', 'f3meanf1', 'f1STD', 'f2STD']]
#49
X =prosody_data[['duration', 'energy', 'power', 'min_pitch', 'max_pitch', 'mean_pitch', 'pitch_sd', 'pitch_abs', 'pitch_quant', 'pitchUvsVRatio', 'Time:8', 'iDifference', 'diffPitchMaxMin', 'diffPitchMaxMean', 'diffPitchMaxMode', 'intensityMin', 'intensityMax', 'intensityMean', 'intensitySD', 'intensityQuant', 'diffIntMaxMin', 'diffIntMaxMean', 'diffIntMaxMode', 'avgVal1', 'avgVal2', 'avgVal3', 'avgBand1', 'avgBand2', 'avgBand3', 'fmean1', 'fmean2', 'fmean3', 'f2meanf1', 'f3meanf1', 'f1STD', 'f2STD', 'f3STD', 'f2STDf1', 'f2STDf2', 'jitter', 'shimmer', 'jitterRap', 'meanPeriod', 'percentUnvoiced', 'numVoiceBreaks', 'PercentBreaks', 'speakRate', 'numPause', 'maxDurPause']]
#25X = prosody_data[['pitch_quant', 'meanPeriod', 'mean_pitch', 'fmean3', 'f3STD', 'f2STD', 'pitch_sd', 'shimmer', 'avgBand2', 'intensityMean', 'jitter', 'intensitySD', 'intensityQuant', 'jitterRap', 'f1STD', 'avgVal2', 'fmean2', 'PercentBreaks', 'AvgTotFall:3', 'f2STDf1', 'pitch_abs', 'f2STDf2', 'avgBand1', 'percentUnvoiced', 'AvgTotRis:3']]
#X = prosody_data.iloc[:, 4:].values



In [3]:
from scipy import linalg as scipy_linalg

In [4]:
def mahalanobis(x=None, data=None, cov=None):
    """Compute the Mahalanobis Distance between each row of x and the data  
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    cov  : covariance matrix (p x p) of the distribution. If None, will be computed from data.
    """
    x_minus_mu = x - np.mean(data)
    if not cov:
        cov = np.cov(data.values.T)
    inv_covmat = scipy_linalg.inv(cov)
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return mahal.diagonal()

In [5]:
df_x =prosody_data[['duration', 'energy', 'power', 'min_pitch', 'max_pitch', 'mean_pitch', 'pitch_sd', 'pitch_abs', 'pitch_quant', 'pitchUvsVRatio', 'Time:8', 'iDifference', 'diffPitchMaxMin', 'diffPitchMaxMean', 'diffPitchMaxMode', 'intensityMin', 'intensityMax', 'intensityMean', 'intensitySD', 'intensityQuant', 'diffIntMaxMin', 'diffIntMaxMean', 'diffIntMaxMode', 'avgVal1', 'avgVal2', 'avgVal3', 'avgBand1', 'avgBand2', 'avgBand3', 'fmean1', 'fmean2', 'fmean3', 'f2meanf1', 'f3meanf1', 'f1STD', 'f2STD', 'f3STD', 'f2STDf1', 'f2STDf2', 'jitter', 'shimmer', 'jitterRap', 'meanPeriod', 'percentUnvoiced', 'numVoiceBreaks', 'PercentBreaks', 'speakRate', 'numPause', 'maxDurPause']]

df_x['mahala'] = mahalanobis(x=df_x, data=prosody_data[['duration', 'energy', 'power', 'min_pitch', 'max_pitch', 'mean_pitch', 'pitch_sd', 'pitch_abs', 'pitch_quant', 'pitchUvsVRatio', 'Time:8', 'iDifference', 'diffPitchMaxMin', 'diffPitchMaxMean', 'diffPitchMaxMode', 'intensityMin', 'intensityMax', 'intensityMean', 'intensitySD', 'intensityQuant', 'diffIntMaxMin', 'diffIntMaxMean', 'diffIntMaxMode', 'avgVal1', 'avgVal2', 'avgVal3', 'avgBand1', 'avgBand2', 'avgBand3', 'fmean1', 'fmean2', 'fmean3', 'f2meanf1', 'f3meanf1', 'f1STD', 'f2STD', 'f3STD', 'f2STDf1', 'f2STDf2', 'jitter', 'shimmer', 'jitterRap', 'meanPeriod', 'percentUnvoiced', 'numVoiceBreaks', 'PercentBreaks', 'speakRate', 'numPause', 'maxDurPause']])
#df_x =X

#df_x['mahala'] = mahalanobis(x=X, data=X)
X=df_x

<ipython-input-5-f4bb98b640ec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['mahala'] = mahalanobis(x=df_x, data=prosody_data[['duration', 'energy', 'power', 'min_pitch', 'max_pitch', 'mean_pitch', 'pitch_sd', 'pitch_abs', 'pitch_quant', 'pitchUvsVRatio', 'Time:8', 'iDifference', 'diffPitchMaxMin', 'diffPitchMaxMean', 'diffPitchMaxMode', 'intensityMin', 'intensityMax', 'intensityMean', 'intensitySD', 'intensityQuant', 'diffIntMaxMin', 'diffIntMaxMean', 'diffIntMaxMode', 'avgVal1', 'avgVal2', 'avgVal3', 'avgBand1', 'avgBand2', 'avgBand3', 'fmean1', 'fmean2', 'fmean3', 'f2meanf1', 'f3meanf1', 'f1STD', 'f2STD', 'f3STD', 'f2STDf1', 'f2STDf2', 'jitter', 'shimmer', 'jitterRap', 'meanPeriod', 'percentUnvoiced', 'numVoiceBreaks', 'PercentBreaks'

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=23, stratify=y)
y_train


array([2, 1, 0, 0, 0, 2, 1, 2, 0, 2, 2, 2, 0, 1, 0, 2, 1, 2, 2, 0, 0, 1,
       1, 2, 0, 0, 0, 1, 2, 0, 1, 1, 2, 1, 0, 1, 1, 1, 0, 0, 1, 2, 0, 1,
       2, 2, 1, 2, 0, 2, 1, 2, 1, 1, 2, 0, 0, 0, 0, 0, 1, 2, 2, 0, 2, 0,
       1, 0, 2, 0, 2, 1, 0, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 0, 1,
       0, 0, 1, 1, 2, 1, 2, 2, 0, 2, 1, 2, 1, 1, 1, 2, 0, 1, 2, 1, 1, 1,
       2, 2, 0, 1, 0, 1, 1, 0, 0, 0, 0, 2, 0, 1, 1, 2, 0, 1, 2, 1, 2, 2,
       1, 0, 0, 1, 0, 2, 1, 2, 0, 1, 1, 0, 0, 0, 2, 2, 1, 2, 0, 2, 2, 2,
       1, 0, 1, 1, 0, 2, 0, 0, 1, 0, 0, 2, 0, 1, 2, 0, 2, 1, 0, 0, 0, 2,
       2, 0, 0, 0, 2, 1, 1, 0, 2, 0, 1, 1, 0, 0, 2, 2, 2, 2, 0, 2, 0, 1,
       0, 2, 1, 1, 1, 0, 1, 0, 1, 2, 1, 2, 2, 1, 2, 0, 0, 1, 1, 1, 0, 2,
       0, 2, 1, 1, 1, 1, 2, 2, 0, 0, 2, 2, 0, 1, 1, 0, 2, 0, 2, 1, 1, 2,
       0, 1, 2, 1, 2, 0, 2, 0, 1, 1, 2, 2, 2, 2, 0, 1, 0, 0, 2, 0, 0, 1,
       1, 2, 0, 1, 2, 1, 0, 1, 1, 0, 0, 1, 1, 2, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 2, 2, 1, 1, 0, 0, 2, 0, 2, 2, 0, 0, 0,

In [7]:
y_01_test = []
for i in y_test:
    if(i == 0):
        y_01_test.append(0)
    else:
        y_01_test.append(1)
#y_01_test

In [8]:
y_01_train = []
for i in y_train:
    if(i == 0):
        y_01_train.append(0)
    else:
        y_01_train.append(1)
#y_01_train

In [42]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
X = X_train
y = y_01_train
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X, y)
clf.score(X, y)
#AdaBoostClassifier(n_estimators=100, random_state=0)
y_01_predict=clf.predict(X_test)
target_names = ['class 0', 'class 1']      
print(classification_report(y_01_test,y_01_predict,target_names=target_names, zero_division=0))

              precision    recall  f1-score   support

     class 0       0.71      0.65      0.68        23
     class 1       0.83      0.87      0.85        45

    accuracy                           0.79        68
   macro avg       0.77      0.76      0.76        68
weighted avg       0.79      0.79      0.79        68



In [110]:
#select from X_train,y_train where y value is not 0
#X_12_train= []
#y_12_train= []
#for (i,j) in zip(range(len(X_train)),y_train):
#    if(j != 0):
#        X_12_train.append(X_train.iloc[i])
#        y_12_train.append(j)
#y_12_train

#select from X_test,y_test where y value is not 0
#X_12_test= []
#y_12_test= []
#for (i,j) in zip(range(len(X_test)),y_test):
#    if(j != 0):
#        X_12_test.append(X_test.iloc[i])
#        y_12_test.append(j)

X_12_train = X_train.loc[y_train!= 0]
X_12_test = X_test.loc[y_test!= 0]


In [111]:
#select from y_train where y value is not 0
y_12_train= []
for i in y_train:
    if(i != 0):
        y_12_train.append(i)
#y_12_train

In [112]:
#select from y_test where y value is not 0
y_12_test = []
for i in y_test:
    if(i != 0):
        y_12_test.append(i)
#y_12_test

In [113]:
X = X_12_train
y = y_12_train
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X, y)
clf.score(X, y)
y_12_predict = clf.predict(X_12_test)
target_names = ['class 1', 'class 2']      
print(classification_report(y_12_test,y_12_predict ,target_names=target_names, zero_division=0))

              precision    recall  f1-score   support

     class 1       0.67      0.55      0.60        22
     class 2       0.63      0.74      0.68        23

    accuracy                           0.64        45
   macro avg       0.65      0.64      0.64        45
weighted avg       0.65      0.64      0.64        45



In [118]:
print(list(y_12_predict))
print(list(y_01_predict))
print(list(y_test))

[1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1]
[1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 2, 0, 1, 0, 0, 2, 0, 2, 2, 0, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 0, 2, 1, 0, 1, 0, 1, 1, 2, 2, 1, 1, 0, 1, 2, 2, 2, 0, 2, 1, 1, 2, 0, 2, 0, 2, 0, 0, 1, 0, 2, 2, 2, 0, 0, 1, 0, 2, 1, 2, 0, 0, 0, 0, 1, 1]


In [121]:
y1=list(y_test)
y2=list(y_01_predict)
y3=list(y_12_predict)
combined_Predictions = []
k=0
for (i,j) in zip(y1,y2):
    if(i!=0):
        combined_Predictions.append(y3[k])
        k=k+1
    else:
        combined_Predictions.append(j)
    
print(combined_Predictions)

[1, 1, 1, 1, 0, 1, 2, 0, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 1, 0, 2, 2, 2, 2, 1, 1, 0, 2, 1, 2, 2, 0, 2, 2, 1, 2, 0, 1, 0, 2, 0, 0, 2, 1, 1, 2, 2, 0, 1, 2, 0, 2, 2, 2, 1, 0, 0, 0, 1, 1]


In [122]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_test,combined_Predictions ,target_names=target_names, zero_division=0))

              precision    recall  f1-score   support

     class 0       1.00      0.65      0.79        23
     class 1       0.46      0.55      0.50        22
     class 2       0.63      0.74      0.68        23

    accuracy                           0.65        68
   macro avg       0.70      0.65      0.66        68
weighted avg       0.70      0.65      0.66        68

